In [12]:
# import necessary libraries
import pandas as pd
from flask import Flask, render_template
# %%
# Module used to connect Python with MongoDb
from flask_pymongo import PyMongo
import datetime
import pprint as pp
import pymongo
# Import Scrape libraries
import requests
import urllib.request
import time 
from splinter import Browser
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
 


# Definitions

 # Set up Mongo Data Base and Populate with test code
 Mars_db is the Databasename
 MData is the collections, or table name, where we will store
 all of the data we scrape and store in variables.
 

In [25]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# declare database
db = client.mars_db  # implicitly declared mars DB

collection = db.mars_db
collection.drop() # clean it out before we add the test data
post = {'nasatitle': 'CORGIES ROAMING MARS!!',
'nasanote': 'In a surprise development today  a pack of what appear to be cute Corgies were seen roaming an Ancient Alien ruin on Mars. This has completely rattled the scientfic community. Nay sayers claim it is just another example of fake news, but Corgie lovers world-wide say it is proof that Corgies are the best and brightest dog breed.',
'nasaimgURL': 'https://media1.giphy.com/media/vGo2sgzeC8r60/200w.webp?cid=790b76116174feb42843815998a66504ca992d6761f09bc0&rid=200w.webp',
'nasacardimage': '',
'nasadate': datetime.datetime.utcnow(),
'datatable1': "",
'datatable2':"",
'jpl_featured_image_url': 'https://media1.giphy.com/media/wOJhrjx3lFxh9ShCZY/giphy.webp?cid=790b761122897cf1a00f010852e522814f44989b84eb167c&rid=giphy.webp',
'mars_weather': 'Partly dusty, 70 degrees F',
'popups':[],
'hemispherepics': [{'Title': 'Water on Mars', 'hemisphere_url': 'https://media1.giphy.com/media/fnlXXGImVWB0RYWWQj/200w.webp?cid=790b76116174feb42843815998a66504ca992d6761f09bc0&rid=200w.webp'},
{'Title': 'Anti-Gravity Technology',
    'hemisphere_url': 'https://media1.giphy.com/media/wOJhrjx3lFxh9ShCZY/giphy.webp?cid=790b761122897cf1a00f010852e522814f44989b84eb167c&rid=giphy.webp'},
{'Title': 'Advanced Linguistics',
    'hemisphere_url': 'https://media3.giphy.com/media/WEiKBTaESHHhK/200w.webp?cid=790b76116174feb42843815998a66504ca992d6761f09bc0&rid=200w.webp'},
{'Title': 'Martian Corgi Spacesuit', 'hemisphere_url': 'https://media2.giphy.com/media/RuP38JSfj14je/giphy.webp?cid=790b76116174feb42843815998a66504ca992d6761f09bc0&rid=giphy.webp'}]
}
collection.insert_one(post)
# Verify results:
results = collection.find_one()
# for result in results:
#     print(result)
results

{'_id': ObjectId('5d73f3cd11ef67c1ba649043'),
 'nasatitle': 'CORGIES ROAMING MARS!!',
 'nasanote': 'In a surprise development today  a pack of what appear to be cute Corgies were seen roaming an Ancient Alien ruin on Mars. This has completely rattled the scientfic community. Nay sayers claim it is just another example of fake news, but Corgie lovers world-wide say it is proof that Corgies are the best and brightest dog breed.',
 'nasaimgURL': 'https://media1.giphy.com/media/vGo2sgzeC8r60/200w.webp?cid=790b76116174feb42843815998a66504ca992d6761f09bc0&rid=200w.webp',
 'nasacardimage': '',
 'nasadate': datetime.datetime(2019, 9, 7, 18, 15, 41, 322000),
 'datatable1': '',
 'datatable2': '',
 'jpl_featured_image_url': 'https://media1.giphy.com/media/wOJhrjx3lFxh9ShCZY/giphy.webp?cid=790b761122897cf1a00f010852e522814f44989b84eb167c&rid=giphy.webp',
 'mars_weather': 'Partly dusty, 70 degrees F',
 'popups': [],
 'hemispherepics': [{'Title': 'Water on Mars',
   'hemisphere_url': 'https://media1

# Set up web pages to scrape as a dictionary
This makes is easier to address the 

In [26]:
def SetSources():
    URL_DataSources={}
    '''Assigns URL addresses to Dictionary'''
    URL_DataSources.update({'NASA':'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'})
    URL_DataSources.update({'JPL':'https://www.jpl.nasa.gov/spaceimages/'})
    URL_DataSources.update({'Mars Weather': 'https://twitter.com/MarsWxReport'})
    URL_DataSources.update({'Mars Facts': 'https://space-facts.com/mars/'})
    URL_DataSources.update({'Hemispheres': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'})
    return URL_DataSources
URLS=SetSources()
pp.pprint(URLS)   

{'Hemispheres': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars',
 'JPL': 'https://www.jpl.nasa.gov/spaceimages/',
 'Mars Facts': 'https://space-facts.com/mars/',
 'Mars Weather': 'https://twitter.com/MarsWxReport',
 'NASA': 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'}


In [27]:
URLNames= URLS.keys()
print(URLNames)

dict_keys(['NASA', 'JPL', 'Mars Weather', 'Mars Facts', 'Hemispheres'])


In [28]:
# response= requests.get(URLS['NASA'] ) 
# soup = bs(response.text, 'lxml')#"html.parser")
# tags = soup.find_all('div')
# print(soup.prettify())

# Scrape Logic for  NASA website
It turned out that Inspect did not clearly show the critical 'rollover_description_inner' attribute
but is was clear when the soup was printed

In [29]:
# download s the HTML NASA Webpage
response= requests.get(URLS['NASA'] ) 
soup = bs(response.text, 'lxml')#"html.parser")
allcontents = soup.find_all('div', "content_title") 
allteasers = soup.find_all('div','rollover_description_inner')
retimage=urljoin(URLS['NASA'], soup.find('img',class_='list_image'))
#retlistdate = soup.find_one('div',class_='list_date')
#allnasaimages = soup.find_one('div', "list_image"  )
#alldates = soup.find_one('div', "list_date"  )
#     pp.pprint(allcontents)
 
#print(allteasers)
#print(allnasaimages[0].text)
#print(alldates[0].text)
pp.pprint(allcontents[0].text.strip())
pp.pprint(allteasers[0].text.strip())
print(retimage)
print(soup.find_all('div','rollover_description_inner')[0].text.strip())

'NASA Invites Students to Name Mars 2020 Rover'
('Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay '
 "contest to name NASA's next Mars rover.")
https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest
Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


# Scrap Logic for JPL Site


In [18]:
from urllib.parse import urljoin # used to concactenate base address with relative image address for total address

response= requests.get(URLS['JPL'] )
soup = bs(response.text, 'lxml')
 

bigpicofday=soup.find_all('a', 'fancybox', id='full_image' ) 
strstr=str(bigpicofday[0])#convert string to string for some pythonic reason
strt =(strstr.find('data-fancybox-href'))
lsubstr=len('data-fancybox-href="')#include the quote
#print(strt,lsubstr)
newstr=strstr[lsubstr+strt:-2].split(" ")[0]  
#print(newstr)

base = URLS['JPL']
print(base)
newstr=urljoin(base, newstr)
newstr=newstr[0:len(newstr)-1]
print(newstr)
print(urljoin(base, newstr))
#pp.pprint(bigpicofday[0])

https://www.jpl.nasa.gov/spaceimages/
https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16842_ip.jpg
https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16842_ip.jpg


# Logic for scraping Mars Weather Data


In [19]:
response= requests.get(URLS['Mars Weather'] )
soup = bs(response.text, 'lxml')
#retweather = soup.find_all('a',id='Report') #,'js-tweet-text-container')
retweather = soup.find('div','js-tweet-text-container').text #,'js-tweet-text-container')
print(retweather )


We won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/ …pic.twitter.com/fWruE2v151



# Logic for Scraping Mars Facts

In [20]:
response= requests.get(URLS['Mars Facts'] )
soup = bs(response.text, 'lxml') 
retdata= soup.find('table', id='tablepress-p-mars')  
retstrdata=(retdata)

#df=parse_html_table(retstrdata) # routine I scraped from web, though there is pandas way
dftablelist=pd.read_html(URLS['Mars Facts'])
dftablelist[0] 

URLS['Mars Facts']

'https://space-facts.com/mars/'

In [21]:
dftablelist[1].columns=['Parameter Description','Value']
dftablelist[1]

,Parameter Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Convert data frames to HTML
For insertion into web Page

In [22]:
rethtmltable0=dftablelist[0].to_html
rethtmltable1=dftablelist[1].to_html
print(rethtmltable1)

<bound method DataFrame.to_html of   Parameter Description                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers>


# Logic for scraping the Hemipheres Web Page

In [23]:
 
response= requests.get(URLS['Hemispheres'] )
soup = bs(response.text, 'lxml') 
# This gives us the four blocks of data associated with each picture
retpics = soup.find_all('div', class_='item')
retpics

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/><div class="description"><h3>Cerberus Hemisphere Enhanced</h3><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></a></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png"/><div class="description"><h3>Schiaparelli Hemisphere En

In [24]:
    #Each of the above blocks needs to be scraped of an image href, a description I will use as a popup, and a url to where the image is, on aonther web page
    popups=[]
    imgurls=[]
    for item in retpics:
            popups.append(item.text)
            imgurls.append(urljoin(URLS['Hemispheres'],item.find('a')['href']))
    imgurls                        

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

## URLS for *where* the images are has been established in imgurls
Now I need to find out where the image is for each URL

In [41]:
rethemiimgs =[]
rethemicaptions =[]
rethemisubcaptions =[]
rethemidict=[]

for item in imgurls:
    time.sleep(1)
    response= requests.get(item)
    soup = bs(response.text, 'lxml')
    image=urljoin(item, soup.find('img',class_='wide-image').get('src'))
    rethemiimgs.append(image)
    titles=soup.title.text.split('|')
    rethemicaptions.append(titles[0].replace(' Enhanced ',""))
    rethemisubcaptions.append(titles[1].strip())
    rethemidict.append({'title':titles[0],'img_url':image})


print(rethemidict)

[{'title': 'Cerberus Hemisphere Enhanced ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


# Collected code for single call that will scrape all data from the five websites

In [42]:
def dprint(flag,str):
    '''
    Purpose:Debug Print
    flag:set to True if output is desired
    str: item to print
    '''
    if flag:
        print(str)
def dpprint(flag,str):
    '''
    Purpose:Debug Print
    flag:set to True if output is desired
    str:item to print
    '''
    if flag:
        pp.pprint(str)
printflag=False
for n in URLNames:
    if n =='NASA':
        # Connect to the URL using urllib request
        response= requests.get(URLS[n] )
        soup = bs(response.text, 'lxml')
        allcontents = soup.find_all('div', "content_title" )
        allteasers = soup.find_all('div','rollover_description_inner'   )
        retimage=urljoin(URLS['NASA'], soup.find('img',class_='list_image'))
        retcontent= allcontents[0].text.strip()
        retteaser=  allteasers[0].text.strip()
        dprint(printflag,retcontent)
        dprint(printflag,retteaser)
    elif n == 'JPL':
        response= requests.get(URLS[n] )
        soup = bs(response.text, 'lxml')        
        bigpicofday=soup.find_all('a', 'fancybox', id='full_image' ) 
        strstr=str(bigpicofday[0])#convert string to string for some pythonic reason
        strt =(strstr.find('data-fancybox-href'))
        lsubstr=len('data-fancybox-href="')#include the quote
        newstr=strstr[lsubstr+strt:-1].split(" ")[0]  
        newstr=urljoin(URLS['JPL'], newstr)
        newstr=newstr[0:len(newstr)-1]
        retpicofday=(urljoin(URLS['JPL'], newstr)) 
        dprint(printflag,"JPL Pic of day: "+retpicofday)
    elif n=='Mars Weather':
        response= requests.get(URLS['Mars Weather'] )
        soup = bs(response.text, 'lxml')    
        retweather = soup.find('div','js-tweet-text-container').text #,'js-tweet-text-container')
        dprint(printflag,retweather )
    elif n=='Mars Facts':
        response= requests.get(URLS['Mars Facts'] )
        soup = bs(response.text, 'lxml') 
#         retdata= soup.find('table', id='tablepress-p-mars')  
#         dprint(printflag,retdata)
        
#         dftablelist=pd.read_html(URLS['Mars Facts'])
#         dftablelist[0]
#         dftablelist[1].columns=['Parameter Description','Value']
#         dftablelist[1]
#         rethtmltable0=dftablelist[0].to_html
#         rethtmltable1=dftablelist[1].to_html
#         dpprint(printflag,rethtmltable0)
#         dprint(printflag,"")
#         dpprint(printflag,rethtmltable1)
 
        retdata= soup.find_all('table')
        rethtmltable0=retdata[0]
        rethtmltable1=retdata[1]
    elif n=='Hemispheres':
        response= requests.get(URLS['Hemispheres'] )
        soup = bs(response.text, 'lxml') 
        
        # This gives us the four blocks of data associated with each picture
        retpics = soup.find_all('div', class_='item')
        
        #Each of the above blocks needs to be scraped of an image href, a description I will use as a popup, and a url to where the image is, on aonther web page
        popups=[]
        imgurls=[]
        
        #loop throught the web pages, get popup text for pictures and get webpage of picture
        for item in retpics:
            popups.append(item.text)
            imgurls.append(urljoin(URLS['Hemispheres'],item.find('a').get('href')))
            
        rethemiimgs =[] # return list of image addresses
        rethemicaptions =[] # Caption to place under image
        rethemisubcaptions =[] #Source of image caption
        for item in imgurls:
            time.sleep(1)
            response= requests.get(item)
            soup = bs(response.text, 'lxml')
            image=urljoin(item, soup.find('img',class_='wide-image').get('src'))
            rethemiimgs.append(image)
            titles=soup.title.text.split('|')
            rethemicaptions.append(titles[0].replace(' Enhanced ',""))
            rethemisubcaptions.append(titles[1].strip())
            rethemidict.append({'title':titles[0],'img_url':image})
        dprint(printflag,rethemicaptions)
        dprint(printflag,rethemisubcaptions)
        dpprint(printflag,rethemiimgs) 
        dprint(printflag,rethemidict)

# Put scraped data into Mongo DB Mdata
User return value variable 

In [43]:
post = {'nasatitle': retcontent,
'nasanote': retteaser,
'nasaimgURL': retimage,
'nasacardimage': retimage,
'nasadate': datetime.datetime.utcnow(),
'datatable1': str(rethtmltable0),
'datatable2':str(rethtmltable1),
'jpl_featured_image_url': retpicofday,
'mars_weather': retweather,
'popups':popups,
'hemispherepics':rethemidict
}
collection.insert_one(post)
# Verify results:

results = collection.find()[1]
# for result in results:
#     print(result)
#results = db.Mdata.find().limit(1).sort({natural:-1})
#results = db.Mdata.find({}).sort({ _id: -1 }).limit(1)

 
print(type(results))
print(results)
# def printmongodict(item):
#     for item in results:
       
#         if type(item)=='<class dict>':
#             printmongodict(item)
#         else:
#             print(results[item])
# printmongodict(results)            
    

<class 'dict'>
{'_id': ObjectId('5d72ef1f96b84e91b4f8aaa0'), 'nasatitle': 'NASA Invites Students to Name Mars 2020 Rover', 'nasanote': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.", 'nasaimgURL': 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest', 'nasacardimage': 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest', 'nasadate': datetime.datetime(2019, 9, 6, 23, 43, 27, 536000), 'datatable1': '<table class="tablepress tablepress-id-comp-mars blue-table" id="tablepress-comp-mars"><thead><tr class="row-1"><th class="column-1"><strong>Mars - Earth Comparison</strong></th><th class="column-2"><span class="mars">Mars</span></th><th class="column-3"><span class="earth">Earth</span></th></tr></thead><tbody><tr class="row-2"><td class=